## data_cleaning.ipynb

Performs data cleaning for restaurant reviews collected via web scraping. It includes extracting structured information, handling missing values, checking for duplicates, and preparing the data for further analysis.


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from src import cleaning

### Select the raw data to process

In [ ]:
raw_data_path = '../data/raw/'

name = 'hd'
reviews_raw = pd.read_csv(raw_data_path + 'collected_reviews_' + name + '.csv')
resumme_raw = pd.read_csv(raw_data_path + 'resumme_' + name + '.csv')
display(resumme_raw)
display(reviews_raw.sample(5))

### Search words selected

Define a dictionary of regular expressions to extract specific fields (service, meal type, price range, scores, etc.) from the review text.

In [ ]:
restaurant_search_words = {
    'service': r'Servicio\n([^\n]+)',
    'meal_type': r'Tipo de comida\n([^\n]+)',
    'price_per_person': r'Precio por persona\n([0-9€\- ]+)',
    'food_score': r'Comida: (\d+)',
    'service_score': r'Servicio: (\d+)',
    'atmosphere_score': r'Ambiente: (\d+)',
    'recommended': r'Platos recomendados\n([^\n]+)'
}

In [ ]:
reviews = reviews_raw.copy()

### Removing duplicates

Check for duplicated rows in the dataset and remove them to ensure data integrity

In [ ]:
# Convert any list-like columns to strings so they can be checked for duplicates
check_dups = reviews.copy()
for col in check_dups.columns:
    if check_dups[col].dtype == 'object' and isinstance(check_dups[col].iloc[0], list):
        check_dups[col] = check_dups[col].apply(lambda x: str(x))

# Now you can check and remove duplicates
duplicates_count = check_dups.duplicated().sum()
print(f"Number of duplicated rows: {duplicates_count}")

# Remove duplicates
reviews.drop_duplicates(inplace=True)
print("Duplicates removed successfully.")

### Prepare and process all fields

Clean and convert relevant columns to numeric types, extract additional details (e.g., average price per person), and drop unnecessary columns from the DataFrame.

In [ ]:
reviews['local_guide_reviews'] = reviews['local_guide_info'].apply(cleaning.extractReviewCount)
reviews['rating_score'] = reviews['rating'].apply(cleaning.extractStarRating)
reviews = cleaning.applyExtractDetails(reviews, search_words = restaurant_search_words)
reviews['recommendations_list'] = reviews['recommended'].apply(cleaning.extractRecommendations)
reviews['date'] = reviews['date_text'].apply(cleaning.convertToDate)

reviews['food_score'] = reviews['food_score'].apply(pd.to_numeric, errors='coerce')
reviews['service_score'] = reviews['service_score'].apply(pd.to_numeric, errors='coerce')
reviews['atmosphere_score'] = reviews['atmosphere_score'].apply(pd.to_numeric, errors='coerce')
reviews['avg_price_per_person'] = reviews['price_per_person'].str.extract(r'-(\d+)\s*€')
reviews['avg_price_per_person'] = pd.to_numeric(reviews['avg_price_per_person'], errors='coerce').astype('Int64')


reviews.drop(columns = ['text_backup', 'local_guide_info', 'rating', 'author', 'recommended', 'date_text'], inplace = True)
reviews.reset_index(inplace=True)
reviews.rename(columns={'index': 'review_id', 'price_per_person':'price_per_person_category'}, inplace=True)

### Check null values

Fill missing values in specific columns with defaults (e.g., 1 for local_guide_reviews, 1 for rating_score).

In [ ]:
# Check for missing values in each column
missing_values = reviews.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Optionally, you can also check the percentage of missing values
missing_percentage = (reviews.isnull().mean() * 100).round(2)
print("Percentage of missing values per column:")
print(missing_percentage)


In [ ]:
# Fill NA values
reviews['local_guide_reviews'] = reviews['local_guide_reviews'].fillna(1)
reviews['rating_score'] = reviews['rating_score'].fillna(1)

### Variables distribution

Generate a summary of the numeric variables in the dataset. This provides insights into the distribution of ratings, review counts, and prices.

In [ ]:
# Summary of numeric columns
print("Summary of numeric variables:")
display(reviews.describe())

# Summary of categorical columns
print("Distribution of categorical variables:")
for col in reviews.select_dtypes(include=['object']).columns:
    if col in ("review", 'recommendations_list', 'date'):
        continue
    print(f"\n{col} distribution:")
    print(reviews[col].value_counts())


### Saving clean data to processed folder

In [ ]:
csv_file_path = '../data/processed/'
reviews.to_csv(csv_file_path + name + '_reviews.csv', index=False)
print('OK! -> processed reviews saved at', csv_file_path + name + '_reviews.csv')

In [ ]:
display(reviews.sample(20))